In [70]:
# Loading the data file from local download
path_fastText = 'wiki-news-300d-1M.vec'
dictionary = open(path_fastText, 'r', encoding='utf-8',
                  newline='\n', errors='ignore')
embeds = {}
for line in dictionary:
    tokens = line.rstrip().split(' ')
    embeds[tokens[0]] = [float(x) for x in tokens[1:]]
    
    if len(embeds) == 100000:
        break

In [77]:
import pandas as pd
import numpy as np

path_to_train = 'tweets.csv'
training = pd.read_csv(path_to_train, encoding='latin-1')[['v1', 'v2']]
text = training['v2']
label = training['v1']

path_to_test = 'test_tweet.csv'
testing = pd.read_csv(path_to_test, encoding='latin-1')[['v1', 'v2']]
tweet = testing['v2']
user = testing['v1']
training[2:5]
testing[0:2]


,v1,v2
0,usr1,Thank you to Great Republican @SenJohnKennedy ...
1,usrt,Great progress has been made by NATO over the ...


In [79]:
from keras.preprocessing.text import text_to_word_sequence
array_length = 20 * 300
embedding_features = pd.DataFrame()
for document in text:
    # Saving the first 20 words of the document as a sequence
    words = text_to_word_sequence(document)[0:20] 
    
    # Retrieving the vector representation of each word and 
    # appending it to the feature vector 
    feature_vector = []
    for word in words:
        try:
            feature_vector = np.append(feature_vector, 
                                       np.array(embeds[word]))
        except KeyError:
            # In the event that a word is not included in our 
            # dictionary skip that word
            pass
    # If the text has less then 20 words, fill remaining vector with
    # zeros
    zeroes_to_add = array_length - len(feature_vector)
    feature_vector = np.append(feature_vector, 
                               np.zeros(zeroes_to_add)
                               ).reshape((1,-1))
    
    # Append the document feature vector to the feature table
    embedding_features = embedding_features.append( 
                                     pd.DataFrame(feature_vector))

In [80]:
testing_features = pd.DataFrame()
for document in tweet:
    # Saving the first 20 words of the document as a sequence
    words = text_to_word_sequence(document)[0:20] 
    
    # Retrieving the vector representation of each word and 
    # appending it to the feature vector 
    feature_vector = []
    for word in words:
        try:
            feature_vector = np.append(feature_vector, 
                                       np.array(embeds[word]))
        except KeyError:
            # In the event that a word is not included in our 
            # dictionary skip that word
            pass
    # If the text has less then 20 words, fill remaining vector with
    # zeros
    zeroes_to_add = array_length - len(feature_vector)
    feature_vector = np.append(feature_vector, 
                               np.zeros(zeroes_to_add)
                               ).reshape((1,-1))
    
    # Append the document feature vector to the feature table
    testing_features = testing_features.append( 
                                     pd.DataFrame(feature_vector))

In [82]:
print(embedding_features.shape)

(258, 6000)


In [83]:
print(testing_features.shape)

(2, 6000)


In [85]:

from sklearn.svm import LinearSVC

embeded_model = LinearSVC()
embeded_model.fit(embedding_features, 
                  label)
embeded_prediction = embeded_model.predict(
                   testing_features)


In [86]:
embeded_prediction

array(['@realDonaldTrump', '@realDonaldTrump'], dtype=object)